In [2]:
import pandas as pd
import json
import time
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
from geopy.geocoders import Nominatim

buydata_df = pd.read_json('buy_data.json')

In [ ]:
buydata_df

In [3]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []        # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []         # 8
address = []               # 9
geo_code = []               # 10
law = []
floor = []
district = []
province_city = []
type = []
type_estate = []
image_links = []

In [ ]:
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
import pandas as pd

buydata_df["geo_code"] = [None for _ in range(len(buydata_df))]

def get_geocode(address):
    api_key = 'b48b7a0d92434d7ba9e13d0feb2db0ad'
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"

    response = requests.get(url)
    data = response.json()

    if data['results']:
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
        print(f"Latitude: {latitude}, Longitude: {longitude}")
        return [latitude, longitude]
    else:
        # Check if address contains "Đường"        
        if "Đường" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Đường ", "").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)  
        elif "Dự án" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Dự án ", "").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)      
        else:
            print("No results found.")
            return None

def data_normalization(df):
    for i, address in enumerate(df["address"]):
        df.at[i, "geo_code"] = get_geocode(address)
        district, province_city = extract_district_and_province(address)
        df.at[i, "district"] = district.replace('Huyện ', '').replace('Quận ', '').replace('TP ', '').strip()            
        df.at[i, "province_city"] = province_city.replace('.','').strip()
        
    for i, price in enumerate(df['price']):
        if isinstance(price, str):           
            df.at[i, 'price'] = price.replace(' tỷ', '').replace(',', '.').strip()    
            
    for i, toilet in enumerate(df['toilet']):
        if isinstance(toilet, str):
         df.at[i, 'toilet'] = toilet.replace(' phòng', '').strip()
        
    for i, bedroom in enumerate(df['bedroom']):        
        if isinstance(bedroom, str):
            df.at[i, 'bedroom'] = bedroom.replace(' phòng', '').strip()
    
    for i, area in enumerate(df['area']):
        if isinstance(area, str):
            df.at[i, 'area'] = area.replace(' m', '').replace('²', '').replace(',', '.').strip()    
    
    for i, width in enumerate(df['width']):        
        if isinstance(width, str):
            df.at[i, 'width'] = width.replace(' m', '').strip() 
                  
    for i, floor in enumerate(df['floor']):        
        if isinstance(floor, str):
            df.at[i, 'floor'] = floor.replace(' tầng', '').strip() 
            
    for i,type_estate in enumerate(df['type_estate']):        
        if isinstance(type_estate, str):
            df.at[i, 'type_estate'] = type_estate.replace('Bán ', '').strip()         
        
def extract_district_and_province(address):
    parts = address.split(',')
    if len(parts) >= 2:
        district = parts[-2].strip()
        province_city = parts[-1].strip()
        return district, province_city
    return None, None

data_normalization(buydata_df)

buydata_df['type_estate'] = buydata_df['type_estate'].replace({
    'nhà mặt phố': 'Nhà Mặt Phố',
    'nhà biệt thự, liền kề': 'Nhà Biệt Thự, Liền Kề',
    'nhà riêng': 'Nhà Đất Thổ Cư',
    'căn hộ chung cư': 'Căn Hộ Chung Cư',
    'căn hộ chung cư mini': 'Căn Hộ Chung Cư',
    'đất nền dự án': 'Đất Nền Dự Án',
    'đất': 'Đất Nền Dự Án',
    'trang trại, khu nghỉ dưỡng': 'Đất Trang Trại',
    'condotel': 'Condotel',
    'văn phòng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'shophouse, nhà phố thương mại': 'ShopHouse',
    'cửa hàng, ki ốt': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'kho, nhà xưởng, đất': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'kho, nhà xưởng': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'nhà trọ, phòng trọ': 'Nhà Trọ, Phòng Trọ',
})

def write_data(df, field, data_list):
    if not isinstance(data_list, list):
        data_list = []  # Initialize as empty list if not already
    
    if field in df.columns:
        for value in df[field]:
            data_list.append(value)
    else:
        print(f"Error: '{field}' column not found in the DataFrame.")
    
# Populate lists with data from DataFrame
write_data(buydata_df, "title", title)
write_data(buydata_df, "price", price)
write_data(buydata_df, "area", area)
write_data(buydata_df, "type", type)
write_data(buydata_df, "type_estate", type_estate)
write_data(buydata_df, "image_links", image_links)
write_data(buydata_df, "bedroom", bedroom)
write_data(buydata_df, "description", description)
write_data(buydata_df, "width", width)
write_data(buydata_df, "direction", direction)
write_data(buydata_df, "toilet", toilet)
write_data(buydata_df, "address", address)
write_data(buydata_df, "law", law)
write_data(buydata_df, "floor", floor)
write_data(buydata_df, "geo_code", geo_code)
write_data(buydata_df, "district", district)
write_data(buydata_df, "province_city", province_city)

# Display the updated DataFrame
print("\nUpdated DataFrame:")

In [5]:
buydata_df

,title,address,price,area,type,description,bedroom,toilet,direction,law,floor,width,type_estate,image_links,geo_code,district,province_city
0,Căn hộ Vinhomes Grand Park cạnh Vincom trả 375...,"Dự án Vinhomes Grand Park, Phường Long Thạnh M...",2.07,47,Mua Bán,"Xem nhà ngay, đã có nhà thực tế để xemChủ đầu ...",2,1,null,Sổ đỏ/ Sổ hồng,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,None,9,Hồ Chí Minh
1,Em NViên CĐT Gamudaland Celadon City-giá mới t...,"Dự án Celadon City, Đường N1, Phường Sơn Kỳ, T...",5.05,97,Mua Bán,Em là Anh Thư nhân viên trực tiếp chủ đầu tư G...,2,null,null,Hợp đồng mua bán,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[11.39867, 107.39976]",Tân Phú,Hồ Chí Minh
2,Căn hộ cao cấp sông Hàn - Giá gốc CĐT - 52 tri...,"Dự án Peninsula Đà Nẵng, Phường Nại Hiên Đông ...",3.7,72,Mua Bán,Dự án được mong chờ nhất 2024 - Dự án Peninsul...,2,null,null,Sổ đỏ/ Sổ hồng,null,null,Căn Hộ Chung Cư,https://i1.ytimg.com/vi/0v8mMKOhT9s/hqdefault.jpg,"[15.65, 108.26667]",Sơn Trà,Đà Nẵng
3,Mở bán quỹ căn góc VIP shophouse Little Hong K...,"Dự án The Empire - Vinhomes Ocean Park 2, Xã L...",17.1,124,Mua Bán,- Little HongKong vị trí cửa ngõ biển tạo sóng...,6,4,Đông - Nam,Hợp đồng mua bán,5,null,"Nhà Biệt Thự, Liền Kề",https://file4.batdongsan.com.vn/resize/1275x71...,"[10.3722, 106.27272]",Văn Giang,Hưng Yên
4,Độc quyền 15 căn ngoại giao - Giá rẻ cọc luôn ...,"Dự án A La Carte Hạ Long Bay, Phường Hùng Thắn...",2.4,50,Mua Bán,"Em Thu Hường - Cư dân dự án À La Carte, hiện đ...",1,1,null,Sổ đỏ/ Sổ hồng,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[22.09797, 105.24147]",Hạ Long,Quảng Ninh
5,Cực hiếm mở bán quỹ căn Thời Đại 68m2 - ôm côn...,"Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...",6.5,60,Mua Bán,Mở bán quỹ căn Thời Đại 68m² - Dãy duy nhất dự...,null,null,null,null,5,null,"Nhà Biệt Thự, Liền Kề",https://file4.batdongsan.com.vn/resize/1275x71...,"[20.94927, 105.98331]",Văn Giang,Hưng Yên
6,VHOCP2 C/K sock 26% - Rẻ hơn hàng cắt lỗ - tổn...,"Dự án The Empire - Vinhomes Ocean Park 2, Xã L...",7,64,Mua Bán,Hotline: (24/7).Dự án Vinhomes Ocean Park 2.==...,null,null,null,null,5,4,"Nhà Biệt Thự, Liền Kề",https://file4.batdongsan.com.vn/resize/1275x71...,"[10.3722, 106.27272]",Văn Giang,Hưng Yên
7,"Rổ hàng Clear Stock từ CĐT, nhận nhà vào ở 15%...","Dự án Celadon City, Đường N1, Phường Sơn Kỳ, T...",3.88,85,Mua Bán,Gamuda Land.Kính gửi quý Anh Chị khách hàng.Em...,2,2,Tây - Bắc,Hợp đồng mua bán,null,null,Căn Hộ Chung Cư,https://i1.ytimg.com/vi/Er_tKTwIX2M/maxresdefa...,"[11.39867, 107.39976]",Tân Phú,Hồ Chí Minh
8,Chính chủ hiện cần bán 1 căn nhà quận 10 khu v...,"Đường Cách Mạng Tháng Tám, Phường 15, Quận 10,...",7.2,39,Mua Bán,"Nhà góc 2 mặt tiền khu Vip đồng bộ cao tầng, h...",null,null,null,null,null,null,Nhà Đất Thổ Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[10.7926968, 106.6537942]",10,Hồ Chí Minh
9,Nhà phố thương mại (shophouse) Sen Vàng Town m...,"Sen Vàng Town, Đường Quách Điêu, Xã Vĩnh Lộc A...",6.9,65,Mua Bán,- Nhà phố xây sẵn 5 tầng tại mặt tiền đường Qu...,null,null,null,Sổ đỏ/ Sổ hồng.,5,null,Nhà Mặt Phố,https://file4.batdongsan.com.vn/resize/1275x71...,"[10.82296, 106.55998]",Bình Chánh,Hồ Chí Minh


In [ ]:
def evaluate_geocode_accuracy(df):
    total_addresses = len(df)
    successful_geocodes = df['geo_code'].count()

    success_rate = (successful_geocodes / total_addresses) * 100
    failure_rate = 100 - success_rate

    print(f"Total addresses: {total_addresses}")
    print(f"Successful geocodes: {successful_geocodes} addresses ({success_rate:.2f}% success)")
    print(f"Failed geocodes: {total_addresses - successful_geocodes} addresses ({failure_rate:.2f}% failure)")

# Evaluate geocoding accuracy for your DataFrame
evaluate_geocode_accuracy(buydata_df)

In [6]:
buydata_df.to_csv('buy_data.csv', encoding='utf-8', index=False)

In [7]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Mua Bán' AND (pi.content LIKE '%tdongsan.com%' OR pi.content LIKE '%ytimg.com%')
        """)
        connection.commit()

finally:
    connection.close()


In [ ]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('buy_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    district = row['district']
                    province_city = row['province_city']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip() or bedroom.lower() == 'null':
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip() or floor.lower() == 'null':
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()
